In [1]:
SBOX = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]
RCON = [0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]


In [2]:
def sub_byte(byte):
    row=byte>>4
    col=byte & 0X0F
    index = (row << 4) + col
    return SBOX[index]



def sub_word(temp):
    out=[0]*4

    for i in range(4):
        out[i]=sub_byte(temp[i])
    return out

def rot_word(temp):
    out=[0]*4

    for i in range(4):
        out[i] = temp[(i + 1) % 4]
    return out
def add_round_key(state,round_key):

    for i in range(4):
        for j in range(4):
            state[i][j]=state[i][j] ^round_key[j][i]
    return state


def key_expanssion(key_bytes):
    
    w=[]
     
    for i in range(4):
        word=[]
        for j in range(4):
            word.append(key_bytes[4 *i+j])
        w.append(word)
    
    for i in range(4, 4*(10+1)):
        temp = [w[i-1][j] for j in range(4)]
        if i% 4==0:
            temp=rot_word(temp)
            temp=sub_word(temp)
            temp[0]^=RCON[i//4]
        new_word=[0]*4
        for j in range(4):
            new_word[j]=w[i-4][j] ^ temp[j]

        w.append(new_word)
    return w

def sub_bytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j]=sub_byte(state[i][j])

    return state

def shift_rows(state):
    
    for i in range(1,4):
        row_values=[state[i][j]for j in range(4)]
        new_row=[0]*4
        for j in range(4):
            new_row[j]=row_values[(i+j) % 4]
        for j in range(4):
            state[i][j]=new_row[j]
    return state

def gmul(a,b):
    p=0
    aa=a
    bb=b
    for i in range(8):
        if(bb&1)==1:
            p^=aa
        high_bit_set=(aa & 0X80)!=0
        aa=(aa<<1) & 0XFF
        if high_bit_set:
            aa^=0X1B
        bb>>=1
    return p & 0XFF






def mix_single_column(col):
    a0,a1,a2,a3=col[0],col[1],col[2],col[3]
    b0=gmul(a0,2)^gmul(a1,3)^a2^a3
    b1=a0^gmul(a1,2)^gmul(a2,3)^a3
    b2=a0^a1^gmul(a2,2)^gmul(a3,3)
    b3=gmul(a0,3)^a1^a2^gmul(a3,2)
    return [b0 & 0xFF, b1 & 0xFF, b2 & 0xFF, b3 & 0xFF]

def mix_column(state):
        for i in range(4):
            col=[state[j][i]for j in range(4)]
            mixed=mix_single_column(col)
            for j in range(4):
                state[j][i]=mixed[j]
        return state
            

def byte_to_state(plaintext):
    state=[[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            state[i][j]=plaintext[i+4*j]
    return state

def aes_encryption(plaintext,key):
    
    state=byte_to_state(plaintext)
    expanded=key_expanssion(key)

    #initial round
    round_0_key=[expanded[i]for i in range(0,4)]

    state=add_round_key(state,round_0_key)

    for i in range(1,10):
        state=sub_bytes(state)
        state=shift_rows(state)
        state=mix_column(state)
        round_key=[expanded[4*i+j]for j in range(4)]
        state=add_round_key(state,round_key)

    state=sub_bytes(state)
    state=shift_rows(state)
    round_key=[expanded[40+i] for i in range(4)]
    state=add_round_key(state,round_key)
    return state

def state_to_byte(state):
    out=[]
    for i in  range(4):
        for j in range(4):
            out.append(state[j][i] & 0XFF)
    return bytes(out)

        
      

plaintext = b"HELLO_AES_WORLd@" 
key = b"MY_SECRET_128KEY"

cipher=aes_encryption(plaintext,key)
cipher=state_to_byte(cipher)
print("plaintext=",plaintext.hex())
print("Cipher Text =",cipher.hex())

plaintext= 48454c4c4f5f4145535f574f524c6440
Cipher Text = d62871d8ae88764bc0bbc17198644a53


In [3]:
# ======== AES DECRYPTION CODE ========

# Inverse S-box (you can import or define manually)
INV_SBOX = [
    # 0     1     2     3     4     5     6     7     8     9     A     B     C     D     E     F
    0x52,0x09,0x6A,0xD5,0x30,0x36,0xA5,0x38,0xBF,0x40,0xA3,0x9E,0x81,0xF3,0xD7,0xFB,
    0x7C,0xE3,0x39,0x82,0x9B,0x2F,0xFF,0x87,0x34,0x8E,0x43,0x44,0xC4,0xDE,0xE9,0xCB,
    0x54,0x7B,0x94,0x32,0xA6,0xC2,0x23,0x3D,0xEE,0x4C,0x95,0x0B,0x42,0xFA,0xC3,0x4E,
    0x08,0x2E,0xA1,0x66,0x28,0xD9,0x24,0xB2,0x76,0x5B,0xA2,0x49,0x6D,0x8B,0xD1,0x25,
    0x72,0xF8,0xF6,0x64,0x86,0x68,0x98,0x16,0xD4,0xA4,0x5C,0xCC,0x5D,0x65,0xB6,0x92,
    0x6C,0x70,0x48,0x50,0xFD,0xED,0xB9,0xDA,0x5E,0x15,0x46,0x57,0xA7,0x8D,0x9D,0x84,
    0x90,0xD8,0xAB,0x00,0x8C,0xBC,0xD3,0x0A,0xF7,0xE4,0x58,0x05,0xB8,0xB3,0x45,0x06,
    0xD0,0x2C,0x1E,0x8F,0xCA,0x3F,0x0F,0x02,0xC1,0xAF,0xBD,0x03,0x01,0x13,0x8A,0x6B,
    0x3A,0x91,0x11,0x41,0x4F,0x67,0xDC,0xEA,0x97,0xF2,0xCF,0xCE,0xF0,0xB4,0xE6,0x73,
    0x96,0xAC,0x74,0x22,0xE7,0xAD,0x35,0x85,0xE2,0xF9,0x37,0xE8,0x1C,0x75,0xDF,0x6E,
    0x47,0xF1,0x1A,0x71,0x1D,0x29,0xC5,0x89,0x6F,0xB7,0x62,0x0E,0xAA,0x18,0xBE,0x1B,
    0xFC,0x56,0x3E,0x4B,0xC6,0xD2,0x79,0x20,0x9A,0xDB,0xC0,0xFE,0x78,0xCD,0x5A,0xF4,
    0x1F,0xDD,0xA8,0x33,0x88,0x07,0xC7,0x31,0xB1,0x12,0x10,0x59,0x27,0x80,0xEC,0x5F,
    0x60,0x51,0x7F,0xA9,0x19,0xB5,0x4A,0x0D,0x2D,0xE5,0x7A,0x9F,0x93,0xC9,0x9C,0xEF,
    0xA0,0xE0,0x3B,0x4D,0xAE,0x2A,0xF5,0xB0,0xC8,0xEB,0xBB,0x3C,0x83,0x53,0x99,0x61,
    0x17,0x2B,0x04,0x7E,0xBA,0x77,0xD6,0x26,0xE1,0x69,0x14,0x63,0x55,0x21,0x0C,0x7D
]


def inv_sub_byte(byte):
    row = byte >> 4
    col = byte & 0x0F
    index = (row << 4) + col
    return INV_SBOX[index]


def inv_sub_bytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j] = inv_sub_byte(state[i][j])
    return state


def inv_shift_rows(state):
    for i in range(1, 4):
        row_values = [state[i][j] for j in range(4)]
        new_row = [0]*4
        for j in range(4):
            new_row[j] = row_values[(j - i) % 4]   # opposite direction of shift_rows
        for j in range(4):
            state[i][j] = new_row[j]
    return state


def inv_mix_single_column(col):
    a0, a1, a2, a3 = col
    # Multiply in GF(2^8) by 14, 11, 13, 9 as per AES InvMixColumns matrix
    b0 = gmul(a0, 14) ^ gmul(a1, 11) ^ gmul(a2, 13) ^ gmul(a3, 9)
    b1 = gmul(a0, 9) ^ gmul(a1, 14) ^ gmul(a2, 11) ^ gmul(a3, 13)
    b2 = gmul(a0, 13) ^ gmul(a1, 9) ^ gmul(a2, 14) ^ gmul(a3, 11)
    b3 = gmul(a0, 11) ^ gmul(a1, 13) ^ gmul(a2, 9) ^ gmul(a3, 14)
    return [b0 & 0xFF, b1 & 0xFF, b2 & 0xFF, b3 & 0xFF]


def inv_mix_column(state):
    for i in range(4):
        col = [state[j][i] for j in range(4)]
        mixed = inv_mix_single_column(col)
        for j in range(4):
            state[j][i] = mixed[j]
    return state


def aes_decryption(ciphertext, key):
    state = byte_to_state(ciphertext)
    expanded = key_expanssion(key)

    # Initial round (reverse of final encryption round)
    round_key = [expanded[40 + i] for i in range(4)]
    state = add_round_key(state, round_key)

    for i in range(9, 0, -1):  # rounds 9..1
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)

        round_key = [expanded[4*i + j] for j in range(4)]
        state = add_round_key(state, round_key)
        state = inv_mix_column(state)

    # Final round (reverse of first encryption round)
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)
    round_0_key = [expanded[j] for j in range(4)]
    state = add_round_key(state, round_0_key)

    return state


decrypted_state = aes_decryption(cipher, key)
print(decrypted_state)
decrypted_text = state_to_byte(decrypted_state)
print("Decrypted =", decrypted_text)


[[72, 79, 83, 82], [69, 95, 95, 76], [76, 65, 87, 100], [76, 69, 79, 64]]
Decrypted = b'HELLO_AES_WORLd@'


In [4]:
INV_SBOX = [
    # 0     1     2     3     4     5     6     7     8     9     A     B     C     D     E     F
    0x52,0x09,0x6A,0xD5,0x30,0x36,0xA5,0x38,0xBF,0x40,0xA3,0x9E,0x81,0xF3,0xD7,0xFB,
    0x7C,0xE3,0x39,0x82,0x9B,0x2F,0xFF,0x87,0x34,0x8E,0x43,0x44,0xC4,0xDE,0xE9,0xCB,
    0x54,0x7B,0x94,0x32,0xA6,0xC2,0x23,0x3D,0xEE,0x4C,0x95,0x0B,0x42,0xFA,0xC3,0x4E,
    0x08,0x2E,0xA1,0x66,0x28,0xD9,0x24,0xB2,0x76,0x5B,0xA2,0x49,0x6D,0x8B,0xD1,0x25,
    0x72,0xF8,0xF6,0x64,0x86,0x68,0x98,0x16,0xD4,0xA4,0x5C,0xCC,0x5D,0x65,0xB6,0x92,
    0x6C,0x70,0x48,0x50,0xFD,0xED,0xB9,0xDA,0x5E,0x15,0x46,0x57,0xA7,0x8D,0x9D,0x84,
    0x90,0xD8,0xAB,0x00,0x8C,0xBC,0xD3,0x0A,0xF7,0xE4,0x58,0x05,0xB8,0xB3,0x45,0x06,
    0xD0,0x2C,0x1E,0x8F,0xCA,0x3F,0x0F,0x02,0xC1,0xAF,0xBD,0x03,0x01,0x13,0x8A,0x6B,
    0x3A,0x91,0x11,0x41,0x4F,0x67,0xDC,0xEA,0x97,0xF2,0xCF,0xCE,0xF0,0xB4,0xE6,0x73,
    0x96,0xAC,0x74,0x22,0xE7,0xAD,0x35,0x85,0xE2,0xF9,0x37,0xE8,0x1C,0x75,0xDF,0x6E,
    0x47,0xF1,0x1A,0x71,0x1D,0x29,0xC5,0x89,0x6F,0xB7,0x62,0x0E,0xAA,0x18,0xBE,0x1B,
    0xFC,0x56,0x3E,0x4B,0xC6,0xD2,0x79,0x20,0x9A,0xDB,0xC0,0xFE,0x78,0xCD,0x5A,0xF4,
    0x1F,0xDD,0xA8,0x33,0x88,0x07,0xC7,0x31,0xB1,0x12,0x10,0x59,0x27,0x80,0xEC,0x5F,
    0x60,0x51,0x7F,0xA9,0x19,0xB5,0x4A,0x0D,0x2D,0xE5,0x7A,0x9F,0x93,0xC9,0x9C,0xEF,
    0xA0,0xE0,0x3B,0x4D,0xAE,0x2A,0xF5,0xB0,0xC8,0xEB,0xBB,0x3C,0x83,0x53,0x99,0x61,
    0x17,0x2B,0x04,0x7E,0xBA,0x77,0xD6,0x26,0xE1,0x69,0x14,0x63,0x55,0x21,0x0C,0x7D
]


def invshift_rows(state):
    for i in range(1,4):
        row_val=[state[i][j] for j in range(4)]
        new_row=[0]*4
        for j in range(4):
            new_row[j]=row_val[(j-i)%4]
        for j in range(4):
            state[i][j]=new_row[j]
    return state

def invsub_byte(byte):
    
    row = byte >> 4
    col = byte & 0x0F
    index = (row << 4) + col
    return INV_SBOX[index]

def invsub_bytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j] = invsub_byte(state[i][j])
    return state

def inv_single_mix_column(col):
    a0,a1,a2,a3=col

    b0 = gmul(a0, 14) ^ gmul(a1, 11) ^ gmul(a2, 13) ^ gmul(a3, 9)
    b1 = gmul(a0, 9) ^ gmul(a1, 14) ^ gmul(a2, 11) ^ gmul(a3, 13)
    b2 = gmul(a0, 13) ^ gmul(a1, 9) ^ gmul(a2, 14) ^ gmul(a3, 11)
    b3 = gmul(a0, 11) ^ gmul(a1, 13) ^ gmul(a2, 9) ^ gmul(a3, 14)
    return [b0 & 0xFF, b1 & 0xFF, b2 & 0xFF, b3 & 0xFF]

    

def inv_mix_column(state):
    for i in range(4):
        col=[state[j][i]for j in range(4)]
        mixed=inv_single_mix_column(col)
        for j in range(4):
            state[j][i]=mixed[j]
    
    return state
def aes_decryption_(cipher,key):
    state=byte_to_state(cipher)

    expanded=key_expanssion(key)

    last_round_key=[expanded[40+i]for i in range(4)]
    state=add_round_key(state,last_round_key)

    for i in range(9,0,-1):
        state=invshift_rows(state)
        state=invsub_bytes(state)
        round_key=[expanded[4*i+j]for j in range(4)]
        state=add_round_key(state,round_key)
        state=inv_mix_column(state)


    state=invshift_rows(state)
    state=invsub_bytes(state)
    first_round_key= [expanded[j] for j in range(4)]
    state=add_round_key(state,first_round_key)

    return state
decrypt=aes_decryption_(cipher,key)

decrypt=state_to_byte(decrypt)
print(decrypt)
print("DEcrypt=",decrypt)

b'HELLO_AES_WORLd@'
DEcrypt= b'HELLO_AES_WORLd@'


In [20]:
b=bytes([65,65,77,66])
print(b)

b'AAMB'


RSA

In [47]:
def gcd(a,b):
    while b:
        a,b=b,a%b
    return a

def choose_e(phi):
    e=3
    while e<phi:
        if gcd(e,phi)==1:
            return e
        e+=2
def encryption(e,plaintext,n):
    cipher=[]
    for i in range(len(plaintext)):
        cipher.append(pow(plaintext[i],e,n))
    return cipher

def decryption(cipher,d,n):

    decrypt_massage=bytes ([pow(c,d,n ) for c in cipher])
    return decrypt_massage
p=61
q=53

n=p*q

phi=(p-1)*(q-1)

e=choose_e(phi)


d=pow(e,-1,phi)

plaintext=b"hello world"
cipher=encryption(e,plaintext,n)
cipher_=[cipher[i] & 0xFF for i in range(len(cipher))]
print(bytes(cipher_).hex())

massage=decryption(cipher,d,n)
print(massage)

ecff55556fd65f6f055538
b'hello world'


Deffie Hellman


In [3]:
#global values
p=29
g=5
#private area for Alice
a=6
ya=pow(g,a,p)

#private area for Bob
b=5
yb=pow(g,b,p)

Alice_secret_key=pow(yb,a,p)

Bob_secret_key=pow(ya,b,p)

print("Alice secret key",Alice_secret_key)
print("Bob secret key",Bob_secret_key)

Alice secret key 25
Bob secret key 25
